<a href="https://colab.research.google.com/github/msalmani/test/blob/master/rapids_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rapids on colab

### [0] Checking environment 

In [0]:
!nvidia-smi 

Wed Jul 10 09:06:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!python -V; pip -V

Python 3.6.8
pip 19.1.1 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


In [0]:
!pip install numba

In [0]:
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda/nvvm/libdevice/'

In [0]:
from numba import cuda, vectorize, guvectorize
from numba import void, uint8 , uint32, uint64, int32, int64, float32, float64, f8
import numpy as np 

@cuda.jit('void(float32[:], float32[:], float32[:])')
def cu_multexamp(a, b, c):
    """This kernel function will be executed by a thread."""
    i = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x 
    if i > c.size:
        return
    c[i] = a[i] * b[i]

device = cuda.get_current_device()
n = 1000000
a = np.arange(n, dtype=np.float32)
b = np.arange(n, dtype=np.float32)
c = np.empty_like(a)

# Assign equivalent storage on device
da = cuda.to_device(a)
db = cuda.to_device(b)
# Assign storage on device for output
dc = cuda.device_array_like(a)

tpb = device.WARP_SIZE
bpg = int(np.ceil((n)/tpb))
cu_multexamp[bpg, tpb](da, db, dc)
# Transfer output from device to host
c = dc.copy_to_host()
print (c)

[0.00000e+00 1.00000e+00 4.00000e+00 ... 9.99994e+11 9.99996e+11
 9.99998e+11]


### [1] installing `cudf`

In [0]:
!pip install cudf-cuda100

In [0]:
!cp /usr/local/lib/python3.6/dist-packages/librmm.so .

In [0]:
# setting up the required environments
import os
os.environ['NUMBAPRO_NVVM']='/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE']='/usr/local/cuda-10.0/nvvm/libdevice'

In [0]:
# cudf example

import cudf, io, requests
from io import StringIO

url="https://github.com/plotly/datasets/raw/master/tips.csv"
content = requests.get(url).content.decode('utf-8')

tips_df = cudf.read_csv(StringIO(content))
tips_df['tip_percentage'] = tips_df['tip']/tips_df['total_bill']*100

# display average tip by dining party size
print(tips_df.groupby('size').tip_percentage.mean())

size
1     21.72920154872781
2    16.571919173482893
3    15.215685473711831
4    14.594900639351332
5    14.149548965142023
6    15.622920072028379
Name: tip_percentage, dtype: float64


### [2] Installing `cuml`

In [0]:
# installing the cuml dependencies
!apt install libopenblas-base libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
libopenblas-base is already the newest version (0.2.20+ds-4).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
# installing cuml
!pip install cuml-cuda100

In [0]:
!cp /usr/local/lib/python3.6/dist-packages/libcuml.so /usr/lib64-nvidia/

In [0]:
# example cuml
import cudf
from cuml import DBSCAN

# Create and populate a GPU DataFrame
gdf_float = cudf.DataFrame()
gdf_float['0'] = [1.0, 2.0, 5.0]
gdf_float['1'] = [4.0, 2.0, 1.0]
gdf_float['2'] = [4.0, 2.0, 1.0]

# Setup and fit clusters
dbscan_float = DBSCAN(eps=1.0, min_samples=1)
dbscan_float.fit(gdf_float)

print(dbscan_float.labels_)

0    0
1    1
2    2
dtype: int32
